In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import Dataset, ConcatDataset
from torchvision.transforms import Compose

from data.ImageDataset import ImageDataset
from model import Model, get_device
from network.unet import UNet
from pipelines import Mode
from serde import Experiment
from utils.augmentation import VerticalFlip, Rotate, ColRec, GaussianBlur, GaussianNoise, ZoomIn, ZoomOut


### Create Experiment
Create a new experiment everytime you are training a new model.<br />
For __training__, set the argument __new =True__ <br />
For __predicting__ using a trained model, set the argument __new = False__ (See Predict notebook)

Give a meaningful name for that experiment, such that you can understand what parameters were used for training.
For example in the cell below, the experiment was named as 'Adam_w500' because we will be using the Adam optimiser and 1:500 weight ratio in the loss function.

When you create an experiment, the following experiment folders will be created:
1. data/network_data/Adam_w500 -> Stores the trained weights from each epoch as well as the experiment config file.
2. data/output_data/Adam_w500 -> Stores the video output containing predictions for each frame.
3. data/tensor_board_logs/Adam_w500 -> Stores the Tensorboard data generated during training



In [3]:
# Creating an experiment for training.
exp = Experiment(name='Combination2', new=True, overwrite=True)
cfg_path=exp.params['cfg_path']

### Data Setup

In [4]:
## Train Set

##augmentation_list = Compose([
#                                  VerticalFlip(probability=0.5),
#                                  Rotate(),
#                                  ColRec(),
#                                  GaussianNoise(),
#                                  GaussianBlur(probability=0.2),
#                                  ZoomIn(),
#                                 ])

#augmentation_list = None


#train_dataset = ConcatDataset([
#        ImageDataset(dataset_name='Sim_Lane_Train',size=250,cfg_path=cfg_path,augmentation=augmentation_list),
##         ImageDataset(dataset_name='Real_No_Lane_Train',size=250,  cfg_path=cfg_path),

#    ])


#train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                               batch_size=1,
#                                               shuffle=True, num_workers=4)

## Test Set
#test_dataset =  ImageDataset(dataset_name='Sim_Lane_Test',size=20,cfg_path=cfg_path,mode=Mode.TEST, seed=5)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                              batch_size=1,
#                                              shuffle=False, num_workers=4)



In [5]:
# Train Set

augmentation_list = Compose([
                                  VerticalFlip(probability=0.5),
                                  Rotate(probability=0.5),
                                  ColRec(probability=0.2),
                                  GaussianNoise(probability=0.2),
                                  GaussianBlur(probability=0.2),
                                  ZoomIn(probability=0.5),
                                 ])

#augmentation_list = None


train_dataset = ConcatDataset([
        ImageDataset(dataset_name='Image_set_unaugmented_s1',size=250,cfg_path=cfg_path,augmentation=augmentation_list),
        ImageDataset(dataset_name='Image_set_unaugmented_s2',size=250,cfg_path=cfg_path,augmentation=augmentation_list),
        ImageDataset(dataset_name='Image_set_unaugmented_s3',size=250,cfg_path=cfg_path,augmentation=augmentation_list),
        ImageDataset(dataset_name='Image_set_unaugmented_s4',size=250,cfg_path=cfg_path,augmentation=augmentation_list)
        #,ImageDataset(dataset_name='Real_No_Lane_Train',size=200,  cfg_path=cfg_path, augmentation=augmentation_list)
    ])



train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1,
                                               shuffle=True, num_workers=4)

# Test Set
#test_dataset =  ImageDataset(dataset_name='Sim_Lane_Test',size=20,cfg_path=cfg_path,mode=Mode.TEST, seed=5)
#test_dataset =  ImageDataset(dataset_name='Image_set_test_s5',size=100,cfg_path=cfg_path,mode=Mode.TEST, seed=5)
test_dataset =  ConcatDataset([
        ImageDataset(dataset_name='Image_set_test_s5',size=100,cfg_path=cfg_path,mode=Mode.TEST, seed=5),
        #ImageDataset(dataset_name='Real_No_Lane_Train',size=20,cfg_path=cfg_path,mode=Mode.TEST, seed=5)
    ])
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=1,
                                              shuffle=False, num_workers=4)



### Model Setup

In [6]:
# Initialize the model
model = Model(exp,net=UNet)

In [7]:
#Defime optimiser parameters
optimiser= torch.optim.Adam
optimiser_params = {'lr': 0.0002}


In [8]:
# Weighted Loss function
no_lane_weight = 1
lane_weight = 4000

device = get_device()

# Non lane is class 0 and lane is class 1
weight = torch.FloatTensor([no_lane_weight, lane_weight]).to(device)
loss_func_params ={'weight' : weight}

In [9]:
model.setup_model(optimiser=optimiser,
                  optimiser_params=optimiser_params,
                  loss_function=torch.nn.CrossEntropyLoss,
                  loss_func_params=loss_func_params)

In [10]:
%debug

### Training 

In [11]:
total_epochs = 150

print("\t\t Avg.Loss \t Accuracy \t Lane F1 Score")

for i in range(total_epochs):

    print("Epoch [{}/{}]".format(i+1,total_epochs))

    model.net.train()
    for images,labels in train_loader:
        model.train_step(images,labels)
    model.print_stats(mode= "Train")
    model.save_model(i)

    model.net.eval()
    for image,labels in test_loader:
        model.validation_step(image,labels)
    model.print_stats(mode="Test")
model.get_best_model()


		 Avg.Loss 	 Accuracy 	 Lane F1 Score
Epoch [1/150]
0.639556884766
0.0730654528332
Train 		  0.0649	   63.96%	    7.31%
0.290954589844
0.109330675458
Test 		  0.0482	   29.10%	   10.93%
Epoch [2/150]
0.803985595703
0.316411238825
Train 		  0.0424	   80.40%	   31.64%
0.828811645508
0.337056077528
Test 		  0.0347	   82.88%	   33.71%
Epoch [3/150]
0.761032104492
0.164256363733
Train 		  0.0362	   76.10%	   16.43%
0.898849487305
0.462411807639
Test 		  0.0547	   89.88%	   46.24%
Epoch [4/150]
0.760665893555
0.125453024812
Train 		  0.0348	   76.07%	   12.55%
0.766174316406
0.271257371124
Test 		  0.0243	   76.62%	   27.13%
Epoch [5/150]
0.682922363281
0.21708989526
Train 		  0.0346	   68.29%	   21.71%
0.937347412109
0.581276769325
Test 		  0.0267	   93.73%	   58.13%
Epoch [6/150]
0.89274597168
0.26559398182
Train 		  0.0334	   89.27%	   26.56%
0.914352416992
0.504020500133
Test 		  0.0271	   91.44%	   50.40%
Epoch [7/150]
0.902877807617
0.393867250738
Train 		  0.0313	   90.29%	   39.39%


0.910125732422
0.491978609626
Test 		  0.0070	   91.01%	   49.20%
Epoch [57/150]
0.966552734375
0.422550052687
Train 		  0.0133	   96.66%	   42.26%
0.970947265625
0.749737118822
Test 		  0.0034	   97.09%	   74.97%
Epoch [58/150]
0.953247070312
0.270476190476
Train 		  0.0125	   95.32%	   27.05%
0.940460205078
0.593795544451
Test 		  0.0086	   94.05%	   59.38%
Epoch [59/150]
0.965423583984
0.587099125364
Train 		  0.0135	   96.54%	   58.71%
0.94499206543
0.612740358793
Test 		  0.0031	   94.50%	   61.27%
Epoch [60/150]
0.986419677734
0.311145510836
Train 		  0.0123	   98.64%	   31.11%
0.99137878418
0.909873983091
Test 		  0.0034	   99.14%	   90.99%
Epoch [61/150]
0.898773193359
0.370588235294
Train 		  0.0170	   89.88%	   37.06%
0.986099243164
0.862199364695
Test 		  0.0084	   98.61%	   86.22%
Epoch [62/150]
0.91731262207
0.284053375611
Train 		  0.0140	   91.73%	   28.41%
0.970779418945
0.748654679092
Test 		  0.0032	   97.08%	   74.87%
Epoch [63/150]
0.985717773438
0.619202603743
Trai

0.936553955078
0.538819875776
Train 		  0.0073	   93.66%	   53.88%
0.942031860352
0.60023150584
Test 		  0.0048	   94.20%	   60.02%
Epoch [113/150]
0.985778808594
0.946344271733
Train 		  0.0083	   98.58%	   94.63%
0.993621826172
0.931721659588
Test 		  0.0013	   99.36%	   93.17%
Epoch [114/150]
0.996444702148
0.928854961832
Train 		  0.0076	   99.64%	   92.89%
0.969787597656
0.742321707444
Test 		  0.0024	   96.98%	   74.23%
Epoch [115/150]
0.973754882812
0.815964048791
Train 		  0.0065	   97.38%	   81.60%
0.983337402344
0.839317245438
Test 		  0.0020	   98.33%	   83.93%
Epoch [116/150]
0.99446105957
0.943781942078
Train 		  0.0073	   99.45%	   94.38%
0.969650268555
0.741453269206
Test 		  0.0030	   96.97%	   74.15%
Epoch [117/150]
0.959030151367
0.717576522562
Train 		  0.0083	   95.90%	   71.76%
0.969604492188
0.741164241164
Test 		  0.0034	   96.96%	   74.12%
Epoch [118/150]
0.988464355469
0.918039895924
Train 		  0.0116	   98.85%	   91.80%
0.994094848633
0.936463634871
Test 		  0.

In [12]:
#from clean_dir import clean_exp
#clean_exp(exp.exp_name)


# Delete all model files except the best model

from del_files import del_exp
del_exp(exp.exp_name)

Deleted 149 files from exp: Combination2
